In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import *
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.applications import *
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import plot_model

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')

if physical_devices != []:
    print("Using GPU")
    for i in physical_devices:
        tf.config.experimental.set_memory_growth(i, True)
else:
    print("Using CPU")
    pass

# Model Architectures

### VGGNet

In [ ]:
def VGGNet(dim, output_neurons, output_activation):
    """
    dim = dimension of input tensor
    output_neurons = number of neurons in output layers i.e. number of classes
    output_activation = activation in output layer
    """
    
    print("\nTRAINING ON Custom VGG MODEL:-")
    
    def block(tensor, conv_reps, n_filters):
        
        """
        x = previous tensor
        n_conv = number of 2D convolution layers in each layer
        n_filters = number of filters in each 2-D convolution layer
        """
        
        x = Conv2D(filters = n_filters, kernel_size = (3,3), padding = 'same')(tensor)
        x = LeakyReLU()(x)
        
        for i in range(conv_reps-1):
            x = Conv2D(filters = n_filters, kernel_size = (3,3), padding = 'same')(x)
            x = LeakyReLU()(x)
            
        x = MaxPooling2D(pool_size = (2,2), strides = 2, padding = 'same')(x)
        
        return x

    # input-layer
    input = Input(shape = dim)
        
    # feature-extraction    
    k = 64
    x = block(input, conv_reps = 2, n_filters = k)
    x = block(x, conv_reps = 2, n_filters = 2*k)
    x = block(x, conv_reps = 3, n_filters = 4*k)
    x = block(x, conv_reps = 3, n_filters = 8*k)
    x = block(x, conv_reps = 3, n_filters = 16*k)
    
    # 3D -> 1D
    x = Flatten()(x)
    
    # fc-layers
    for i in range(2):
        x = Dense(4096)(x)
        x = LeakyReLU()(x)
    
    # output-layer
    output = Dense(output_neurons, output_activation)(x)  
    
    # MODEL
    model = Model(inputs = input, outputs = output)
    
    return model

### MobileNet

In [ ]:
def MobileNet(dim, output_neurons, output_activation):
    """
    dim = dimension of input tensor
    output_neurons = number of neurons in output layers i.e. number of classes
    output_activation = activation in output layer
    """
    
    print("\nTRAINING ON Custom MobileNet MODEL:-")
    
    def block(x, n_filters, d_strides):
        
        """
        x = previous tensor
        n_filters = number of filters in each 2-D convolution layer
        d_strides = stride in depthwise convolution layer
        """
        
        # depthwise
        x = DepthwiseConv2D(kernel_size = (3,3), strides = d_strides, padding = 'same')(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
    
        # pointwise
        x = Conv2D(filters = n_filters, kernel_size = (1,1))(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        
        return x
    
    # input-layer
    input = Input(shape = dim)
    
    # feature-extraction
    x = Conv2D(filters = 32, kernel_size = (3,3), strides = 2, padding = 'same')(input)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    
    x = block(x, n_filters = 64, d_strides = 1)
    x = block(x, n_filters = 128, d_strides = 2)
    x = block(x, n_filters = 128, d_strides = 1)
    x = block(x, n_filters = 256, d_strides = 2)
    x = block(x, n_filters = 256, d_strides = 1)
    x = block(x, n_filters = 512, d_strides = 2)
    
    for i in range(5):
        x = block(x, n_filters = 512, d_strides = 1)
        
    x = block(x, n_filters = 1024, d_strides = 2)
    x = block(x, n_filters = 1024, d_strides = 1)
    
    # 3D -> 1D
    x = GlobalAveragePooling2D()(x)
    
    # output-layer
    output = Dense(output_neurons, output_activation)(x)  
    
    # MODEL
    model = Model(inputs = input, outputs = output)
    
    return model

### Inception

In [ ]:
def Inception(dim, output_neurons, output_activation):
    """
    dim = dimension of input tensor
    output_neurons = number of neurons in output layers i.e. number of classes
    output_activation = activation in output layer
    """
    
    print("\nTRAINING ON Inception MODEL:-")
    
    def block(x, filters):
        """
        x = previous tensor
        filters = number of filters in each 2-D convolution layer
        """
        
        t1 = Conv2D(filters[0], kernel_size = (1,1))(x)
        t1 = LeakyReLU()(t1)
        
        t2 = Conv2D(filters[1], kernel_size = (1,1))(x)
        t2 = LeakyReLU()(t2)
        t2 = Conv2D(filters[2], kernel_size = (3,3), padding = 'same')(t2)
        t2 = LeakyReLU()(t2)
        
        t3 = Conv2D(filters[3], kernel_size = (1,1))(x)
        t3 = LeakyReLU()(t3)
        t3 = Conv2D(filters[4], kernel_size = (5,5), padding = 'same')(t3)
        t3 = LeakyReLU()(t3)
        
        t4 = MaxPool2D(pool_size = (3,3), strides = 1, padding = 'same')(x)
        t4 = Conv2D(filters[5], kernel_size = (1,1))(t4)
        t4 = LeakyReLU()(t4)
        
        output = Concatenate()([t1, t2, t3, t4])
        
        return output
        
    # input-layer
    input = Input(shape = dim)
    
    x = Conv2D(filters = 64, kernel_size = (7,7), strides = 2, padding = 'same')(input)
    x = LeakyReLU()(x)
    x = MaxPool2D(pool_size = (3,3), strides = 2, padding = 'same')(x)
    
    x = Conv2D(filters = 64, kernel_size = (1,1))(x)
    x = LeakyReLU()(x)
    x = Conv2D(filters = 192, kernel_size = (3,3), strides = 1, padding = 'same')(x)
    x = LeakyReLU()(x)
    x = MaxPool2D(pool_size = (3,3), strides = 2, padding = 'same')(x)
    
    x = block(x, [64, 96, 128, 16, 32, 32])
    x = block(x, [128, 128, 192, 32, 96, 64])
    x = MaxPool2D(pool_size = (3,3), strides = 2, padding = 'same')(x)
    
    x = block(x, [192, 96, 208, 16, 48, 64])
    x = block(x, [160, 112, 224, 24, 64, 64])
    x = block(x, [128, 128, 256, 24, 64, 64])
    x = block(x, [112, 144, 288, 32, 64, 64])
    x = block(x, [256, 160, 320, 32, 128, 128])
    x = MaxPool2D(pool_size = (3,3), strides = 2, padding = 'same')(x)

    x = block(x, [256, 160, 320, 32, 128, 128])
    x = block(x, [384, 192, 384, 48, 128, 128])
    
    # 3D -> 1D
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.4)(x)
    
    # output-layer
    output = Dense(output_neurons, output_activation)(x)
    
    # MODEL
    model = Model(inputs = input, outputs = output)    
    
    return model

### ResNet50

In [ ]:
def ResNet50(dim, output_neurons, output_activation):
    """
    dim = dimension of input tensor
    output_neurons = number of neurons in output layers i.e. number of classes
    output_activation = activation in output layer
    """
    
    print("\nTRAINING ON ResNet50 MODEL:-")
    
    def conv_batchnorm_leakyrelu(x, filters, kernel_size, strides=1):
        """
        x = previous tensor
        filters = number of filters in each 2-D convolution layer
        kernel_size = kernel_size
        strides = stride in convolution layer
        """
        
        x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same')(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)

        return x
    
    def projection_block(tensor, filters, strides):
        """
        tensor = previous tensor
        filters = number of filters in each 2-D convolution layer
        strides = stride in convolution layer
        """
    
        #left stream
        x = conv_batchnorm_leakyrelu(tensor, filters=filters, kernel_size=(1,1), strides=1)
        x = conv_batchnorm_leakyrelu(x, filters=filters, kernel_size=(3,3), strides=strides)
        x = Conv2D(filters=4*filters, kernel_size=(1,1), strides=1)(x)
        x = BatchNormalization()(x)

        #right stream
        shortcut = Conv2D(filters=4*filters, kernel_size=(1,1), strides=strides)(tensor)
        shortcut = BatchNormalization()(shortcut)

        x = Add()([shortcut, x])
        x = LeakyReLU()(x)

        return x
    
    def identity_block(tensor, filters):
        """
        tensor = previous tensor
        filters = number of filters in each 2-D convolution layer
        """
        
        x = conv_batchnorm_leakyrelu(tensor, filters=filters, kernel_size=(1,1), strides=1)
        x = conv_batchnorm_leakyrelu(x, filters=filters, kernel_size=(3,3), strides=1)
        x = Conv2D(filters=4*filters, kernel_size=(1,1), strides=1)(x)
        x = BatchNormalization()(x)

        x = Add()([tensor, x])
        x = LeakyReLU()(x)

        return x
    
    def resnet_block(x, filters, reps, strides):
        """
        x = previous tensor
        filters = number of filters in each 2-D convolution layer
        reps = repeatation of total resnet block [1st rep -> Project; (reps-1) rep -> Identitiy]
        strides = stride in convolution layer
        """
        
        x = projection_block(x, filters, strides)
        for _ in range(reps-1):
            x = identity_block(x, filters)
        return x 
    
    
    # input-layer
    input = Input(shape=dim)
    
    # feature-extraction
    x = conv_batchnorm_leakyrelu(input, filters=64, kernel_size=(7,7), strides=2)
    x = MaxPool2D(pool_size=3, strides=2)(x)

    x = resnet_block(x, filters=64, reps=3, strides=1)
    x = resnet_block(x, filters=128, reps=4, strides=2)
    x = resnet_block(x, filters=256, reps=6, strides=2)
    x = resnet_block(x, filters=512, reps=3, strides=2)

    # 3D->1D
    x = GlobalAveragePooling2D()(x)

    # output-layer
    output = Dense(output_neurons, output_activation)(x)  
    
    # MODEL
    model = Model(inputs = input, outputs = output)

    return model

### DenseNet121

In [ ]:
def DenseNet121(dim, output_neurons, output_activation):
    """
    dim = dimension of input tensor
    output_neurons = number of neurons in output layers i.e. number of classes
    output_activation = activation in output layer
    """
    
    print("\nTRAINING ON DenseNet121 MODEL:-")
    
    def batchnorm_leakyrelu_conv(x, filters, kernel_size):
        """
        x = previous tensor
        filters = number of filters in each 2-D convolution layer
        kernel_size = kernel_size
        """
        
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = Conv2D(filters=filters, kernel_size=kernel_size, padding='same')(x)
        return x
    
    def dense_block(tensor, k, reps):
        """
        tensor = previous tensor
        k = integer coefficient of number of filters
        reps = repeatation of bn_rl_conv block
        """
        
        for _ in range(reps):
            x = batchnorm_leakyrelu_conv(tensor, filters=4*k, kernel_size=1)
            x = batchnorm_leakyrelu_conv(x, filters=k, kernel_size=3)
            tensor = Concatenate()([tensor, x])
        return tensor
    
    def transition_layer(x, theta):
        """
        x = previous tensor
        theta = a fraction by which number of filters should be reduced
        """
        
        f = int(tf.keras.backend.int_shape(x)[-1]*theta)
        x = batchnorm_leakyrelu_conv(x, filters=f, kernel_size=1)
        x = AvgPool2D(pool_size=2, strides=2, padding='same')(x)
        return x 
    
    k = 32
    theta = 0.5
    repetitions = 6, 12, 24, 16

    input = Input(shape=dim)

    x = Conv2D(2*k, 7, strides=2, padding='same')(input)
    x = MaxPool2D(3, strides=2, padding='same')(x)

    for reps in repetitions:
        d = dense_block(x, k, reps)
        x = transition_layer(d,theta)

    x = GlobalAvgPool2D()(d)

    output = Dense(output_neurons, output_activation)(x)  

    model = Model(inputs = input, outputs = output)

    return model

### Xception

In [ ]:
def Xception(dim, output_neurons, output_activation):
    """
    dim = dimension of input tensor
    output_neurons = number of neurons in output layers i.e. number of classes
    output_activation = activation in output layer
    """
    
    print("\nTRAINING ON Xception MODEL:-")
    
    def conv_bn(x, filters, kernel_size, strides=1):
        """
        x = previous tensor
        filters = number of filters in each 2-D convolution layer
        kernel_size = kernel_size
        strides = stride in convolution layer
        """
        
        x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        return x

    def sep_conv_bn(x, filters, kernel_size, strides=1):
        """
        x = previous tensor
        filters = number of filters in each 2-D convolution layer
        kernel_size = kernel_size
        strides = stride in convolution layer
        """
        
        x = SeparableConv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        return x

    def entry_flow(x):
        """
        x = previous tensor
        """
        
        x = conv_bn(x, filters=32, kernel_size=3, strides=2)
        x = ReLU()(x)
        x = conv_bn(x, filters=64, kernel_size=3)
        tensor = ReLU()(x)

        x = sep_conv_bn(tensor, filters=128, kernel_size=3)
        x = ReLU()(x)
        x = sep_conv_bn(x, filters=128, kernel_size=3)
        x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)

        tensor = conv_bn(tensor, filters=128, kernel_size=1, strides=2)

        x = Add()([tensor,x])

        x = ReLU()(x)
        x = sep_conv_bn(x, filters=256, kernel_size=3)
        x = ReLU()(x)
        x = sep_conv_bn(x, filters=256, kernel_size=3)
        x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)

        tensor = conv_bn(tensor, filters=256, kernel_size=1, strides=2)

        x = Add()([tensor,x])

        x = ReLU()(x)
        x = sep_conv_bn(x, filters=728, kernel_size=3)
        x = ReLU()(x)
        x = sep_conv_bn(x, filters=728, kernel_size=3)
        x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)

        tensor = conv_bn(tensor, filters=728, kernel_size=1, strides=2)

        x = Add()([tensor,x])

        return x
    
    def middle_flow(tensor):
        """
        tensor = previous tensor
        """
        
        for _ in range(8):
            x = ReLU()(tensor)
            x = sep_conv_bn(x, filters=728, kernel_size=3)
            x = ReLU()(x)
            x = sep_conv_bn(x, filters=728, kernel_size=3)
            x = ReLU()(x)
            x = sep_conv_bn(x, filters=728, kernel_size=3)
            
            tensor = Add()([tensor,x])

        return tensor       
    
    def exit_flow(tensor):
        """
        tensor = previous tensor
        """
        
        x = ReLU()(tensor)
        x = sep_conv_bn(x, filters=728, kernel_size=3)
        x = ReLU()(x)
        x = sep_conv_bn(x, filters=1024, kernel_size=3)
        x = MaxPool2D(3, strides=2, padding='same')(x)

        tensor = conv_bn(tensor, filters=1024, kernel_size=1, strides=2)

        x = Add()([tensor,x])

        x = sep_conv_bn(x, filters=1536, kernel_size=3)
        x = ReLU()(x)
        x = sep_conv_bn(x, filters=2048, kernel_size=3)
        x = ReLU()(x)    

        x = GlobalAvgPool2D()(x)

        x = Dense(output_neurons, output_activation)(x)  

        return x

    input = Input(shape=dim)

    x = entry_flow(input)
    x = middle_flow(x)
    output = exit_flow(x)

    model = Model(inputs = input, outputs = output)

    return model

# Choose Model Architecture

In [ ]:
h = int(input("Image Dimension(H or W): "))
w = h
dim = (h,w,3)

In [ ]:
output_neurons = int(input("Number of classes: "))

if output_neurons > 1:
    output_activation = 'softmax'
else:
    output_activation = 'sigmoid'

In [ ]:
print("Press 1 for VGGNet")
print("Press 2 for MobileNet")
print("Press 3 for Inception")
print("Press 4 for ResNet50")
print("Press 5 for DenseNet121")
print("Press 6 for Xception")

model_select = int(input("\nChoose model: "))

if model_select == 1:
    model = VGGNet(dim, output_neurons, output_activation)
if model_select == 2:
    model = MobileNet(dim, output_neurons, output_activation)
if model_select == 3:
    model = Inception(dim, output_neurons, output_activation)
if model_select == 4:
    model = ResNet50(dim, output_neurons, output_activation)
if model_select == 5:
    model = DenseNet121(dim, output_neurons, output_activation)
if model_select == 6:
    model = Xception(dim, output_neurons, output_activation)

# Summary

In [ ]:
model.summary()

In [ ]:
plot_model(
    model,
#     to_file="model.png",
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB", # TB -> Vertical; LR -> Horizontal
    expand_nested=False,
    dpi=300,
)